# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Totals
schoolTotal = school_data["school_name"].count()
studentTotal = school_data["size"].sum()
budgetTotal = school_data["budget"].sum()
#Averages
mathAvg = student_data["math_score"].mean()
readingAvg = student_data["reading_score"].mean()
overallAvg = (mathAvg + readingAvg)/2
#Pass rates
mathPass = student_data.loc[student_data["math_score"] >= 70]
mathPerc = (mathPass["math_score"].count() / studentTotal) * 100
readingPass = student_data.loc[student_data["reading_score"] >= 70]
readingPerc = (readingPass["reading_score"].count() / studentTotal) * 100
overallPerc = (mathPerc + readingPerc) / 2
#define data columns and print new df
summary_df = pd.DataFrame([
    {"Total Schools": schoolTotal, "Total Students": studentTotal, "Total Budget" : '${:,.2f}'.format(budgetTotal), \
     "Average Math Score" : mathAvg, "Average Reading Score" : readingAvg, "Overall Average" : overallAvg, \
     "% Pass Math" : mathPerc, "% Pass Reading" : readingPerc, "% Overall Pass Rate" : overallPerc}])
cleansummary_df = summary_df[["Total Schools", "Total Students", "Total Budget", "Average Math Score", \
                                "Average Reading Score", "Overall Average", "% Pass Math", "% Pass Reading", \
                                "% Overall Pass Rate"]]
cleansummary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Overall Average,% Pass Math,% Pass Reading,% Overall Pass Rate
0,15,39170,"$24,649,428.00",78.985371,81.87784,80.431606,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [3]:
#pull csv data into new df
school_df = school_data[["school_name", "type", "size", "budget"]]
school_df["Per Student Budget"] = school_df["budget"] / school_df["size"]
schoolgroup = student_data.groupby(["school_name"])

#merge averages into df
avgMath = round(schoolgroup["math_score"].mean(), 2)
avgRead = round(schoolgroup["reading_score"].mean(), 2)
avg_df = pd.DataFrame({"Average Math Score" : avgMath, "Average Reading Score" : avgRead})
avgMerge = pd.merge(school_df, avg_df, on="school_name")

#merge math and reading percentages into df
passMath = student_data.loc[student_data["math_score"] >= 70, ["school_name", "math_score"]]
groupMath = passMath.groupby(["school_name"])
countMath = groupMath.count()
mathMerge = pd.merge(avgMerge, countMath, on="school_name")
passRead = student_data.loc[student_data["reading_score"] >= 70, ["school_name", "reading_score"]]
groupRead = passRead.groupby(["school_name"])
countRead = groupRead.count()
readMerge = pd.merge(mathMerge, countRead, on="school_name")
readMerge["% Passing Math"] = round((readMerge["math_score"] / readMerge["size"])*100, 2)
readMerge["% Passing Reading"] = round((readMerge["reading_score"] / readMerge["size"])*100, 2)

#merge overall pass rate into df
readMerge["Overall Passing Rate %"] = (readMerge["% Passing Math"] + readMerge["% Passing Reading"]) / 2

#rename columns
rename_df = readMerge.rename(columns={"school_name":"School Name", "type":"School Type", "size":"Total Students", \
                                      "budget":"Total School Budget"})
final_df = rename_df[["School Name", "School Type", "Total Students", "Total School Budget", "Per Student Budget", \
                      "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", \
                      "Overall Passing Rate %"]]
final_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate %
0,Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,73.500
1,Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,73.365
2,Shelton High School,Charter,1761,1056600,600.0,83.36,83.73,93.87,95.85,94.860
3,Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,73.805
4,Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,95.265
5,Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,93.87,96.54,95.205
6,Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,95.585
7,Bailey High School,District,4976,3124928,628.0,77.05,81.03,66.68,81.93,74.305
8,Holden High School,Charter,427,248087,581.0,83.80,83.81,92.51,96.25,94.380
9,Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,95.270


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [4]:
sort_top_df = final_df.sort_values("Overall Passing Rate %", ascending=False)
sort_top_df.reset_index()

,index,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate %
0,6,Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,95.585
1,14,Thomas High School,Charter,1635,1043130,638.0,83.42,83.85,93.27,97.31,95.290
2,9,Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,95.270
3,4,Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,95.265
4,5,Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,93.87,96.54,95.205
5,10,Wright High School,Charter,1800,1049400,583.0,83.68,83.96,93.33,96.61,94.970
6,2,Shelton High School,Charter,1761,1056600,600.0,83.36,83.73,93.87,95.85,94.860
7,8,Holden High School,Charter,427,248087,581.0,83.80,83.81,92.51,96.25,94.380
8,7,Bailey High School,District,4976,3124928,628.0,77.05,81.03,66.68,81.93,74.305
9,3,Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,73.805


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [5]:
sort_worse_df = final_df.sort_values("Overall Passing Rate %")
sort_worse_df.reset_index().head()

,index,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate %
0,11,Rodriguez High School,District,3999,2547363,637.0,76.84,80.74,66.37,80.22,73.295
1,1,Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,73.365
2,0,Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,73.500
3,12,Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,73.640
4,3,Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,73.805


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
#creating grade series
nineMath = student_data.loc[student_data["grade"] == "9th"].groupby(["school_name"])["math_score"]
avgNineMath = round(nineMath.mean(),2)
tenMath = student_data.loc[student_data["grade"] == "10th"].groupby(["school_name"])["math_score"]
avgTenMath = round(tenMath.mean(),2)
elevenMath = student_data.loc[student_data["grade"] == "11th"].groupby(["school_name"])["math_score"]
avgElevenMath = round(elevenMath.mean(),2)
twelveMath = student_data.loc[student_data["grade"] == "12th"].groupby(["school_name"])["math_score"]
avgTwelveMath = round(twelveMath.mean(),2)

#aggregate series into dataframe
math_grade = pd.Series(["Schools", "9th Grade", "10th Grade", "11th Grade", "12th Grade"])
math_grade = pd.DataFrame({"9th Grade" : avgNineMath, "10th Grade" : avgTenMath, "11th Grade" : avgElevenMath, \
                         "12th Grade" : avgTwelveMath})
math_grade.index_name = "school"
math_grade

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
#creating grade series
nineRead = student_data.loc[student_data["grade"] == "9th"].groupby(["school_name"])["reading_score"]
avgNineRead = round(nineRead.mean(),2)
tenRead = student_data.loc[student_data["grade"] == "10th"].groupby(["school_name"])["reading_score"]
avgTenRead = round(tenRead.mean(),2)
elevenRead = student_data.loc[student_data["grade"] == "11th"].groupby(["school_name"])["reading_score"]
avgElevenRead = round(elevenRead.mean(),2)
twelveRead = student_data.loc[student_data["grade"] == "12th"].groupby(["school_name"])["reading_score"]
avgTwelveRead = round(twelveRead.mean(),2)

#aggregate series into dataframe
reading_grade = pd.Series(["Schools", "9th Grade", "10th Grade", "11th Grade", "12th Grade"])
reading_grade = pd.DataFrame({"9th Grade" : avgNineRead, "10th Grade" : avgTenRead, "11th Grade" : avgElevenRead, \
                         "12th Grade" : avgTwelveRead})
reading_grade.index_name = "school"
reading_grade

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [4]:
#scrub for min/max
# print(final_df["Per Student Budget"].max())
# print(final_df["Per Student Budget"].min())
#set bins
bins = [0, 584.9999, 614.9999, 644.9999, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645<"]

#slice data to bins
final_df["Spending Ranges (Per Student)"] = pd.cut(final_df["Per Student Budget"], bins, labels = group_names)
spending_df = final_df.groupby("Spending Ranges (Per Student)")

#find avg math and reading scores
average_math = spending_df["Average Math Score"].mean()
average_read = spending_df["Average Reading Score"].mean()
mathPerc = spending_df["% Passing Math"].mean()
readPerc = spending_df["% Passing Reading"].mean()
overall = (mathPerc + readPerc)/2

score_spend_df = pd.DataFrame(
     {"Average Math Score":average_math, "Average Reading Score":average_read, "% Passing Math":mathPerc, \
      "% Passing Reading":readPerc, "Overall Passing Rate":overall})
score_spend_df

C:\Users\willz\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.452500,83.935000,93.460000,96.610000,95.035000
$585-615,83.600000,83.885000,94.230000,95.900000,95.065000
$615-645,79.078333,81.891667,75.668333,86.106667,80.887500
$645<,76.996667,81.026667,66.163333,81.133333,73.648333


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# Sample bins. Feel free to create your own bins.
# size_bins = [0, 1000, 2000, 5000]
# group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
#scrub for max and min
# print(final_df["Total Students"].max())
# print(final_df["Total Students"].min())

#set bins
size_bins = [0, 999, 1999, 2999, 3999]
size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-3000)", "Very Large (3000<)"]

#slice data to bins
final_df["School Size"] = pd.cut(final_df["Total Students"], size_bins, labels = size_names)
size_df = final_df.groupby("School Size")

#find avg math and reading scores
size_avg_math = size_df["Average Math Score"].mean()
size_avg_read = size_df["Average Reading Score"].mean()
mathPerc = size_df["% Passing Math"].mean()
readPerc = size_df["% Passing Reading"].mean()
overall = (mathPerc + readPerc)/2

size_score_df = pd.DataFrame(
     {"Average Math Score":size_avg_math, "Average Reading Score":size_avg_read, "% Passing Math":mathPerc, \
      "% Passing Reading":readPerc, "Overall Passing Rate":overall})
size_score_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.8200,83.925,93.5500,96.100,94.82500
Medium (1000-2000),83.3740,83.868,93.5980,96.790,95.19400
Large (2000-3000),78.4275,81.770,73.4625,84.475,78.96875
Very Large (3000<),76.8400,80.740,66.3700,80.220,73.29500


## Scores by School Type

* Perform the same operations as above, based on school type.

In [10]:
type_df = final_df.groupby("School Type")
#find avg math and reading scores
type_avg_math = type_df["Average Math Score"].mean()
type_avg_read = type_df["Average Reading Score"].mean()
mathPerc = type_df["% Passing Math"].mean()
readPerc = type_df["% Passing Reading"].mean()
overall = (mathPerc + readPerc)/2

type_score_df = pd.DataFrame(
     {"Average Math Score":type_avg_math, "Average Reading Score":type_avg_read, "% Passing Math":mathPerc, \
      "% Passing Reading":readPerc, "Overall Passing Rate":overall})
type_score_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.472500,83.897500,93.620000,96.586250,95.103125
District,76.955714,80.965714,66.548571,80.798571,73.673571
